In [184]:
!pip install PennyLane

In [185]:
# import libraries
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

In [186]:
# Create a 4-qubit circuit
dev = qml.device("default.qubit", wires=4)

In [187]:
# Create the hidden layers using rotation gates and CNOT gates to train the model.
def layer(W):

    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0) 
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
    qml.Rot(W[2, 0], W[2, 1], W[2, 2], wires=2)
    qml.Rot(W[3, 0], W[3, 1], W[3, 2], wires=3)

    # Using the CNOT gates to make the 4 qubits fully entangled.
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[0, 2])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[1, 3])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[3, 0])

In [188]:
# Create Pauli strings for stabilizer states |0011>, |1010>, |0101>, |1100>.
obs_1 = qml.Identity(0) @ qml.Identity(1) @ qml.Identity(2) @ qml.Identity(3) # iiii
obs_2 = qml.PauliZ(0) @ qml.Identity(1) @ qml.Identity(2) @ qml.Identity(3) # ziii
obs_3 = qml.Identity(0) @ qml.PauliZ(1) @ qml.Identity(2) @ qml.Identity(3) # izii
obs_4 = qml.Identity(0) @ qml.Identity(1) @ qml.PauliZ(2) @ qml.Identity(3) # iizi
obs_5 = qml.Identity(0) @ qml.Identity(1) @ qml.Identity(2) @ qml.PauliZ(3)# iiiz
obs_6 = qml.PauliZ(0) @ qml.PauliZ(1) @ qml.Identity(2) @ qml.Identity(3) # zzii
obs_7 = qml.PauliZ(0) @ qml.Identity(1) @ qml.PauliZ(2) @ qml.Identity(3) # zizi
obs_8 = qml.PauliZ(0) @ qml.Identity(1) @ qml.Identity(2) @ qml.PauliZ(3) # ziiz
obs_9 = qml.PauliZ(0) @ qml.PauliZ(1) @ qml.PauliZ(2) @ qml.Identity(3) # zzzi
obs_10 = qml.PauliZ(0) @ qml.Identity(1) @ qml.PauliZ(2) @ qml.PauliZ(3) # zizz
obs_11 = qml.PauliZ(0) @ qml.PauliZ(1) @ qml.Identity(2) @ qml.PauliZ(3) # zziz
obs_12 = qml.Identity(0) @ qml.PauliZ(1) @ qml.PauliZ(2) @ qml.PauliZ(3) # izzz
obs_13 = qml.Identity(0) @ qml.PauliZ(1) @ qml.Identity(2) @ qml.PauliZ(3) # iziz
obs_14 = qml.Identity(0) @ qml.Identity(1) @ qml.PauliZ(2) @ qml.PauliZ(3) # iizz
obs_15 = qml.Identity(0) @ qml.PauliZ(1) @ qml.PauliZ(2) @ qml.Identity(3) # izzi
obs_16 = qml.PauliZ(0) @ qml.PauliZ(1) @ qml.PauliZ(2) @ qml.PauliZ(3) # zzzz

# The specific states to return given by the task.
output = [[0,0,1,1], [1,0,1,0], [0,1,0,1], [1,1,0,0]] 

# The signs of the Pauli strings.
# out1 is the sign of the Pauli string for the stabilizer state |0011>.
out1 = [1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, 1, -1, 1, -1, 1]
out1 = list(np.array(out1)/16)

# out2 is the sign of the Pauli string for the stabilizer state |1010>.
out2 =[1, -1, 1, -1, 1, -1, 1, -1, 1, 1, -1, -1, 1, -1, -1, 1]
out2 = list(np.array(out2)/16)

# out3 is the sign of the Pauli string for the stabilizer state |0101>.
out3 = [1, 1, -1, 1, -1, -1, 1, -1, -1, -1, 1, 1, 1, -1, -1, 1]
out3 = list(np.array(out3)/16)

# out4 is the sign of the Pauli string for the stabilizer state |1100>.
out4 = [1, -1, -1, 1, 1, 1, -1, -1, 1, -1, 1, -1, -1, 1, -1, 1]
out4 = list(np.array(out4)/16)

# The list of the Pauli strings. 
obs_list = [obs_1, obs_2, obs_3, obs_4, obs_5, obs_6, obs_7, obs_8, obs_9, obs_10, obs_11, obs_12, obs_13, obs_14, obs_15, obs_16]


H_total_1 = qml.Hamiltonian(out1, obs_list)
H_total_2 = qml.Hamiltonian(out2, obs_list)
H_total_3 = qml.Hamiltonian(out3, obs_list)
H_total_4 = qml.Hamiltonian(out4, obs_list)


In [208]:
# Prepare initial input states
def statepreparation(x):
    qml.BasisState(x, wires=[0, 1, 2, 3])
print(qml.BasisState(x, wires=[0, 1, 2, 3]))

NameError: ignored

In [190]:
# Create the quntum variational circuit.
@qml.qnode(dev)
def circuit(weights, x, X_encode):
    statepreparation(x)
    for W in weights:
      layer(W)

    # binary to decimal number
    dec_var=x[0]*(2**3)+x[1]*(2**2)+x[2]*(2**1)+x[3]
    
    # Assign different input generated randomly to specific output given by the task.
    if dec_var == X_encode[0]:
      return qml.expval(H_total_1)
    elif dec_var == X_encode[1]:
      return qml.expval(H_total_2)
    elif dec_var == X_encode[2]:
      return qml.expval(H_total_3)
    elif dec_var == X_encode[3]:
      return qml.expval(H_total_4)

In [191]:
# Return the results from the quantum variational circuit
def variational_classifier(var, x, X_encode):
    weights = var[0]
    return circuit(weights, x, X_encode) 

In [192]:
# Loss function loss = loss + abs(l - p), where l is label and p is fidelity. 
# Use the absolute loss function to rapidly converge the loss.
def abs_loss(labels, fidelity):
    loss = 0
    for l, f in zip(labels, fidelity):
        loss = loss + abs(l - f) 
    loss = loss / len(labels)
    return loss

In [193]:
# Use the average fidelity as the evaluation metrics for the model. 
def avg_fidelity(labels, fidelity):
    loss = 0
    fidelity = sum(fidelity)/4
    return fidelity

In [194]:
# Use the fidelity as the cost function.
def cost(var, X, Y, X_encode):
    fidelity = [variational_classifier(var, x, X_encode) for x in X]
    return abs_loss(Y, fidelity)

In [195]:
from random import random
# Generate random states as the inputs for the quantum variational circuit.
X_random = [[random(),random(),random(),random()],[random(),random(),random(),random()],[random(),random(),random(),random()],[random(),random(),random(),random()]]
for i in range(4):
  for j in range(4):
    if X_random[i][j] < 0.5:
      X_random[i][j] = 0
    else:
      X_random[i][j] = 1

X = X_random

# binary to decimal 
X_encode = [[X_random[0][0]*(2**3)+X_random[0][1]*(2**2)+X_random[0][2]*(2**1)+X_random[0][3]],[X_random[1][0]*(2**3)+X_random[1][1]*(2**2)+X_random[1][2]*(2**1)+X_random[1][3]],[X_random[2][0]*(2**3)+X_random[2][1]*(2**2)+X_random[2][2]*(2**1)+X_random[2][3]],[X_random[3][0]*(2**3)+X_random[3][1]*(2**2)+X_random[3][2]*(2**1)+X_random[3][3]]]

# If the states duplicated, generate a new set of random states. 
while set([x for x in X_encode if X_encode.count(x) > 1]):
  X_random = [[random(),random(),random(),random()],[random(),random(),random(),random()],[random(),random(),random(),random()],[random(),random(),random(),random()]]
  for i in range(4):
    for j in range(4):
      if X_random[i][j] < 0.5:
        X_random[i][j] = 0
      else:
        X_random[i][j] = 1

  X = X_random
  X_encode = [[X_random[0][0]*(2**3)+X_random[0][1]*(2**2)+X_random[0][2]*(2**1)+X_random[0][3]],[X_random[1][0]*(2**3)+X_random[1][1]*(2**2)+X_random[1][2]*(2**1)+X_random[1][3]],[X_random[2][0]*(2**3)+X_random[2][1]*(2**2)+X_random[2][2]*(2**1)+X_random[2][3]],[X_random[3][0]*(2**3)+X_random[3][1]*(2**2)+X_random[3][2]*(2**1)+X_random[3][3]]]

print("X_random:", X_random)

# Label the fidelity of the specific outputs given by the task to be 1. 
Y = [1,1,1,1]

X_random: [[0, 1, 1, 0], [1, 1, 0, 0], [0, 0, 0, 1], [0, 1, 1, 1]]


In [196]:
# Setup the initial variable
np.random.seed(0)
num_qubits = 4 # Number of the qubits used in the quantum variational circuit
num_layers = 8 # Number of the hidden layers.
var_init = (0.01 * np.random.randn(num_layers, num_qubits, 3), 0.0) 
print(var_init)

(tensor([[[ 0.01764052,  0.00400157,  0.00978738],
         [ 0.02240893,  0.01867558, -0.00977278],
         [ 0.00950088, -0.00151357, -0.00103219],
         [ 0.00410599,  0.00144044,  0.01454274]],

        [[ 0.00761038,  0.00121675,  0.00443863],
         [ 0.00333674,  0.01494079, -0.00205158],
         [ 0.00313068, -0.00854096, -0.0255299 ],
         [ 0.00653619,  0.00864436, -0.00742165]],

        [[ 0.02269755, -0.01454366,  0.00045759],
         [-0.00187184,  0.01532779,  0.01469359],
         [ 0.00154947,  0.00378163, -0.00887786],
         [-0.01980796, -0.00347912,  0.00156349]],

        [[ 0.01230291,  0.0120238 , -0.00387327],
         [-0.00302303, -0.01048553, -0.01420018],
         [-0.0170627 ,  0.01950775, -0.00509652],
         [-0.00438074, -0.01252795,  0.0077749 ]],

        [[-0.01613898, -0.0021274 , -0.00895467],
         [ 0.00386902, -0.00510805, -0.01180632],
         [-0.00028182,  0.00428332,  0.00066517],
         [ 0.00302472, -0.00634322, -0.00

In [197]:
# Optimizer for the model (Ref: https://pennylane.readthedocs.io/en/stable/code/api/pennylane.NesterovMomentumOptimizer.html)
opt = NesterovMomentumOptimizer(0.5) 
# batch size 
batch_size = 8

In [198]:
var = var_init # Variables of the rotation gates at the hidden layers

# Training process
for it in range(100):
    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = np.array(X)[batch_index.astype(int)]
    Y_batch = np.array(Y)[batch_index.astype(int)]
    var = opt.step(lambda v: cost(v, X_batch, Y_batch, X_encode), var)
    # Compute accuracy
    fidelity = [variational_classifier(var, x, X_encode) for x in X]
    print(fidelity)
    acc = avg_fidelity(Y, fidelity)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(var, X, Y, X_encode), acc
        )
    )

[5.22432438611331e-06, 0.9997756262823823, 9.578368165646944e-06, 1.0625389738436186e-05]
Iter:     1 | Cost: 0.7500497 | Accuracy: 0.2499503 
[3.939118800484753e-06, 0.999918854556334, 9.638485553797871e-06, 1.0453482527719715e-05]
Iter:     2 | Cost: 0.7500143 | Accuracy: 0.2499857 
[3.2975146444652603e-06, 0.9999757472893547, 1.1037300883198764e-05, 1.1672858684405596e-05]
Iter:     3 | Cost: 0.7499996 | Accuracy: 0.2500004 
[1.8366513531917206e-06, 0.9999714336604906, 1.5734891397792272e-05, 1.6289356777970943e-05]
Iter:     4 | Cost: 0.7499987 | Accuracy: 0.2500013 
[1.1155202870039016e-06, 0.9999291733966993, 2.9164486594818184e-05, 2.971449352322242e-05]
Iter:     5 | Cost: 0.7500027 | Accuracy: 0.2499973 
[6.866673254862787e-07, 0.9998168816981094, 9.249231644087308e-05, 9.316089276409367e-05]
Iter:     6 | Cost: 0.7499992 | Accuracy: 0.2500008 
[4.41125274704135e-07, 0.9996849102335734, 0.00020304069107367206, 0.00020397382432094102]
Iter:     7 | Cost: 0.7499769 | Accuracy: 0